In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

df = pd.read_parquet("finewebC_danish.parquet")

In [ ]:
labels = np.concatenate(df["educational_value_labels"].values)
np.unique(labels)

label_counts = defaultdict(int)
for label in labels:
    label_counts[label] += 1

print(dict(label_counts))

{'None': 1278, 'Minimal': 696, '❗ Problematic Content ❗': 370, 'Basic': 185, 'Good': 34, 'Excellent': 10}


In [ ]:
# Filter out rows with problematic content
def get_sample(df, N, bias):

    filtered_df = df[df["problematic_content_label_present"] == False]
    filtered_df = filtered_df[filtered_df["educational_value_labels"].apply(lambda x: len(x) > 2)]

    mask_ex = filtered_df["educational_value_labels"].apply(lambda labels: any(label in {'Excellent'} for label in labels))
    mask_good = filtered_df["educational_value_labels"].apply(lambda labels: any(label in {'Good'} for label in labels))

    for i in range(len(mask_ex)):
        if np.random.rand() > bias:
            mask_good[i] = False

    
    combined_mask = mask_ex | mask_good
    count = len(filtered_df[combined_mask])
    
    if count >= N:
        return count

    filtered_df_sampling = filtered_df[combined_mask == False]

    df_sample = filtered_df_sampling.sample(n=N - count, random_state=43)

    df_final = pd.concat([filtered_df[combined_mask], df_sample], ignore_index=True)

    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

    return df_final

In [239]:
df_final = get_sample(df, 100, 0.7)

labels = np.concatenate(df_final["educational_value_labels"].values)
label_counts = defaultdict(int)
for label in labels:
    label_counts[label] += 1

print(dict(label_counts))

{'None': 124, 'Basic': 39, 'Minimal': 104, 'Excellent': 9, 'Good': 24}


/var/folders/t1/pvhvgdyj15g798qw31m8tb8c0000gn/T/ipykernel_65083/2042690181.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  count = len(filtered_df[combined_mask])
/var/folders/t1/pvhvgdyj15g798qw31m8tb8c0000gn/T/ipykernel_65083/2042690181.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df_sampling = filtered_df[combined_mask == False]
/var/folders/t1/pvhvgdyj15g798qw31m8tb8c0000gn/T/ipykernel_65083/2042690181.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_final = pd.concat([filtered_df[combined_mask], df_sample], ignore_index=True)


In [ ]:
# df_final.to_csv("data_to_annotate.csv", index=False)